## SALARY REGRESSION

In [ ]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder  
import pickle 

In [3]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
# data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [10]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [11]:
onehot_encoder_geo = OneHotEncoder()
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']])

In [12]:
geo_encoded_df = pd.DataFrame(geo_encoded.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [13]:
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

In [14]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [15]:
X = data.drop('EstimatedSalary',axis=1)
y = data['EstimatedSalary']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [18]:
with open('label_encoder_gender.pickle', 'wb') as file:
    pickle.dump(label_encoder_gender, file)
with open('onehot_encoder_geo.pickle', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)
with open('scaler.pickle', 'wb') as file:
    pickle.dump(scaler, file)

In [19]:
## ANN Training with regression 

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense

In [20]:
model = Sequential()
model.add(Dense(units=64, activation='relu', input_shape=(X_train.shape[1],))) ## hidden layer 1
model.add(Dense(units=32, activation='relu'))         ## hidden layer 2
model.add(Dense(units=1))       ## output layer isliye 1 unit and sigmoid activation fn

In [21]:
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
import datetime
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import tensorflow as tf

log_dir = "regressionLogs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [24]:
early_stop = EarlyStopping(monitor='val_loss',patience=10, restore_best_weights=True)

In [25]:
history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop, tensorboard_callback]
)

Epoch 1/100


250/250 [==============================] - 6s 11ms/step - loss: 100348.4531 - mae: 100348.4531 - val_loss: 98412.9297 - val_mae: 98412.9297
Epoch 2/100
250/250 [==============================] - 2s 7ms/step - loss: 99265.0312 - mae: 99265.0312 - val_loss: 96252.7734 - val_mae: 96252.7734
Epoch 3/100
250/250 [==============================] - 2s 7ms/step - loss: 95546.9609 - mae: 95546.9609 - val_loss: 90878.3906 - val_mae: 90878.3906
Epoch 4/100
250/250 [==============================] - 3s 10ms/step - loss: 88465.4297 - mae: 88465.4297 - val_loss: 82355.4453 - val_mae: 82355.4453
Epoch 5/100
250/250 [==============================] - 4s 14ms/step - loss: 78865.6172 - mae: 78865.6172 - val_loss: 72417.5547 - val_mae: 72417.5547
Epoch 6/100
250/250 [==============================] - 3s 10ms/step - loss: 68737.5625 - mae: 68737.5625 - val_loss: 63282.7812 - val_mae: 63282.7812
Epoch 7/100
250/250 [==============================] - 2s 10ms/step - loss: 60306.3086 - mae: 6030

In [26]:
%load_ext tensorboard

In [29]:
%tensorboard --logdir regressionLogs/fit20260113-213615/

Reusing TensorBoard on port 6006 (pid 15448), started 0:00:55 ago. (Use '!kill 15448' to kill it.)

In [30]:
test_loss, test_mae = model.evaluate(X_test,y_test)
print(f'MAE: {test_mae}')

63/63 [==============================] - 0s 3ms/step - loss: 50341.6406 - mae: 50341.6406
MAE: 50341.640625


In [32]:
model.save('regression.h5')

c:\Users\hp\Dl_Basics\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
